In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from collections import Counter

In [2]:
raw_df = pd.read_csv('288821rows.csv')

C:\Users\Sean\AppData\Local\Temp\ipykernel_10328\627538862.py:1: DtypeWarning: Columns (10,11,63,66,68,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv('288821rows.csv')


In [3]:
df = raw_df[['num_comments', 'title']]

In [7]:
df['tokenized'] = df['title'].apply(word_tokenize)

C:\Users\Sean\AppData\Local\Temp\ipykernel_10328\3888837297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokenized'] = df['title'].apply(word_tokenize)


In [8]:
df

,num_comments,title,tokenized
0,4,What's the best thing to say to someone who's ...,"[What, 's, the, best, thing, to, say, to, some..."
1,8,Where would someone be able to get their hands...,"[Where, would, someone, be, able, to, get, the..."
2,4,What’s your ideal first date?,"[What, ’, s, your, ideal, first, date, ?]"
3,3,"Law Enforcement Agents of Reddit, what do the ...","[Law, Enforcement, Agents, of, Reddit, ,, what..."
4,3,What sounds innocent but also condescending?,"[What, sounds, innocent, but, also, condescend..."
...,...,...,...
288816,8,"Those who decided not to get married, why?","[Those, who, decided, not, to, get, married, ,..."
288817,9,"If one body part could be easily replaceable, ...","[If, one, body, part, could, be, easily, repla..."
288818,29,Those who don’t have/like to use a bidet at ho...,"[Those, who, don, ’, t, have/like, to, use, a,..."
288819,12,How did you find out about / get on Reddit?,"[How, did, you, find, out, about, /, get, on, ..."


In [34]:
vocab = []

for i in df['tokenized']:
    vocab.extend(i)

In [35]:
counts = Counter(vocab)

In [37]:
counts.most_common()

[('?', 306442),
 ('you', 210683),
 ('What', 167319),
 ('the', 127571),
 (',', 88024),
 ('to', 85038),
 ('your', 81264),
 ('a', 81145),
 ('is', 80947),
 ('do', 62058),
 ('of', 59796),
 ('that', 49507),
 ('what', 46417),
 ('in', 46320),
 ('and', 45181),
 ('’', 40595),
 ('are', 38555),
 ('would', 36683),
 ('it', 35864),
 ("'s", 32932),
 ('have', 26726),
 ('for', 26632),
 ('or', 25324),
 ('s', 24960),
 ('be', 24266),
 ('How', 22193),
 ('on', 22096),
 ('most', 20997),
 ('was', 20669),
 ('with', 19686),
 ('thing', 19287),
 ('who', 17763),
 ('.', 17575),
 ('If', 17372),
 ('why', 16999),
 ('about', 16619),
 ('did', 16521),
 ('one', 15086),
 ('how', 14903),
 ('people', 14372),
 ('if', 14132),
 ('something', 14041),
 ('but', 13837),
 ('ever', 13608),
 ('life', 13514),
 ('best', 13351),
 ('like', 13236),
 ('Reddit', 12874),
 ('think', 12592),
 ('when', 12405),
 ('can', 11454),
 ('not', 11369),
 ('from', 11248),
 ('someone', 11077),
 ('an', 10565),
 ('some', 10344),
 ('time', 9993),
 ('had', 9943)

In [38]:
counts_df = pd.DataFrame(counts.most_common(), columns=['word','count'])